In [1]:
# Get access to Google disk
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
# Import libraries
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import matplotlib.pyplot as plt
import pandas as pd
from pyspark.sql.functions import col
from pyspark.sql.functions import to_timestamp

In [3]:
# Создаём SparkSession
spark = SparkSession.builder \
    .appName("PetCo") \
    .getOrCreate()

In [4]:
# # Добавляем файлы  в Spark-контекст
from pyspark import SparkFiles

spark.sparkContext.addFile("/content/drive/MyDrive/Colab Notebooks/diploma/scripts/process_data.py")
spark.sparkContext.addFile("/content/drive/MyDrive/Colab Notebooks/diploma/scripts/tests_for_datasets.py")


import process_data
from tests_for_datasets import test_actions_sasrec, test_items_sasrec, common_tests_sasrec

In [5]:
%cd "/content/drive/MyDrive/Colab Notebooks/diploma/scripts"

/content/drive/MyDrive/Colab Notebooks/diploma/scripts


In [6]:
WORKING_DIRECTORY = '/content/drive/MyDrive/Colab Notebooks/diploma/'

In [7]:
# !jupyter nbconvert --to script process_data.ipynb
# !jupyter nbconvert --to script tests_for_datasets.ipynb

# Приведем датасет с логами юзеров к формату для SASRec

In [8]:
# Read dataset with beh logs
data_logs = spark.read.parquet(WORKING_DIRECTORY+'data/needed_beh_logs')
data_logs.show()

+----------------+--------------------+-----+--------------------+--------------------+--------------------+-----------------+-------------+--------------------+--------------------+-----------+--------------------+------------+--------------------+--------------------+--------------------+-------+
|customer_user_id|              ac_key|ac_id|             user_id|          session_id|           timestamp|         raw_term|  filter_name|        filter_value|              action|customer_id|           item_name|variation_id|      variation_name|               items|            items_v2|revenue|
+----------------+--------------------+-----+--------------------+--------------------+--------------------+-----------------+-------------+--------------------+--------------------+-----------+--------------------+------------+--------------------+--------------------+--------------------+-------+
|       435675978|key_afiSr5Y4gCaaSW5X| 2560|e5fbf045-94fa-498...|e5fbf045-94fa-498...|2024-11-29 10

In [9]:
# Приведем датасет к презентабельному виду
data_actions_cleaned = process_data.clean_data_actions(data_logs, 'conversion')
data_actions_cleaned.show()

Найдено строк c пропущенных значений: 0
+-------+-------+--------------------+-------+
|user_id|item_id|           timestamp|revenue|
+-------+-------+--------------------+-------+
|  -1002|5181461|2024-06-14 15:17:...|  10.69|
|  -1002|5206702|2024-06-14 15:19:...|  16.49|
|  -1002|5119257|2024-06-14 15:21:...|   4.39|
|  -1002|5107211|2024-06-28 19:06:...|  69.48|
|  -1002|5008694|2024-07-01 18:43:...|  59.98|
|  -1002|5181462|2024-07-17 10:11:...|  23.73|
|  -1002|5000991|2024-07-17 10:12:...|  55.99|
|  -1002|5000991|2024-07-17 10:19:...|  55.99|
|  -1002|5080929|2024-07-20 21:55:...|  36.99|
|  -1002|5183021|2024-07-25 17:05:...|  67.99|
|  -1002|5183309|2024-07-30 17:02:...|  68.98|
|  -1002|5142756|2024-08-14 17:22:...|  17.18|
|  -1002|5206288|2024-08-17 14:32:...|  15.88|
|  -1002|5181804|2024-08-21 01:23:...|  19.99|
|  -1002|5118422|2024-09-24 15:09:...|  20.69|
|  -1002|5180971|2024-10-03 15:14:...|  46.99|
|  -1002|5180971|2024-10-03 15:14:...|  46.99|
|  -1002|5180971|202

In [10]:
# Сохраним презентабельный вид датасета
cleaned_data_folder_path = WORKING_DIRECTORY+'data/cleaned_data/'
output_name = 'data_conversions'
process_data.save_dataset_parquet(data_actions_cleaned, cleaned_data_folder_path, output_name)

Папка /content/drive/MyDrive/Colab Notebooks/diploma/data/cleaned_data/ уже существует.
Датасет сохранён по пути /content/drive/MyDrive/Colab Notebooks/diploma/data/cleaned_data/data_conversions.


In [11]:
# Теперь приведем датасет к формату SASRec
data_actions_processed = process_data.actions_to_sasrec_format(data_actions_cleaned)
print(f"Размер датасета: ({data_actions_processed.count()}, {len(data_actions_processed.columns)})")
data_actions_processed.show()

Найдено строк c пропущенных значений: 8366
Колонка 'user_id' содержит 8366 пропущенных значений (NULL).
Найдено строк c пропущенных значений: 8366
Колонка 'user_id' содержит 8366 пропущенных значений (NULL).
Размер датасета: (441616, 4)
+-------+-------+-------------------+------+
|user_id|item_id|           datetime|weight|
+-------+-------+-------------------+------+
|  -1002|5181461|2024-06-14 15:17:25|     1|
|  -1002|5206702|2024-06-14 15:19:15|     1|
|  -1002|5119257|2024-06-14 15:21:21|     1|
|  -1002|5107211|2024-06-28 19:06:10|     1|
|  -1002|5008694|2024-07-01 18:43:40|     1|
|  -1002|5181462|2024-07-17 10:11:22|     1|
|  -1002|5000991|2024-07-17 10:12:30|     1|
|  -1002|5000991|2024-07-17 10:19:52|     1|
|  -1002|5080929|2024-07-20 21:55:22|     1|
|  -1002|5183021|2024-07-25 17:05:36|     1|
|  -1002|5183309|2024-07-30 17:02:31|     1|
|  -1002|5142756|2024-08-14 17:22:13|     1|
|  -1002|5206288|2024-08-17 14:32:32|     1|
|  -1002|5181804|2024-08-21 01:23:02|     1

In [12]:
# Сохраним датасет формата SASRec
sasrec_data_folder_path = WORKING_DIRECTORY+'data/sasrec_format/'
output_name = 'actions'
process_data.save_dataset_parquet(data_actions_processed, sasrec_data_folder_path, output_name)
# data_actions_processed = spark.read.parquet(sasrec_data_folder_path+output_name)

Папка /content/drive/MyDrive/Colab Notebooks/diploma/data/sasrec_format/ уже существует.
Датасет сохранён по пути /content/drive/MyDrive/Colab Notebooks/diploma/data/sasrec_format/actions.


In [13]:
# Проведем тесты на правильность сборки датасета для SASRec
test_actions_sasrec(df_final=data_actions_processed, df_original=data_logs, action='conversion')

✅ В датасете правильные колонки и у них правильные типы данных
✅ В датасете нет пропущенных значений
✅ В датасете правильное число юзеров, которые сделали действие
✅ Для всех юзеров правильно указаны товары и время действий


# Приведем датасет с признаками товаров к формату SASRec

In [11]:
# Read dataset with items
data_items = spark.read.parquet(WORKING_DIRECTORY+'data/data_set_items')
# data_items.show()
# Print metadata and facets columns
process_data.print_metadata(data_items)

Основные метаданные:
url: /shop/en/petcostore/product/kaytee-fortidiet-prohealth-ratmouse-food-5-lbs
mfName: Kaytee
itemimg: https://assets.petco.com/petco/image/upload/f_auto,q_auto,w_190/dpr_auto/1226754-center-1
itemurl: /shop/en/petcostore/product/kaytee-fortidiet-prohealth-ratmouse-food-5-lbs
itemname: Kaytee Forti-Diet Pro Health Mouse, Rat Food
group_ids: ['repeat-delivery-eligible-products', 'rat-food', 'small-animal-repeat-delivery-products', 'same-day-delivery-small-pet-products', 'small-animal-food', 'buy-online-pick-up-in-store-small-animal-products']
image_url: https://assets.petco.com/petco/image/upload/f_auto,q_auto,w_190/dpr_auto/1226754-center-1
startDate: 26297760
UPC_NUMBER: 71859000015
deactivated: False
AverageRating: 4.5395
TotalReviewCount: 76
parentCatEntryID: 865023
PTC_OMNI_TAXONOMY: Consumables|Food|Household Pet
PTC_OMNI_BOPUS_FLAG: Yes
PTC_OMNI_RX_FOOD_IND: No
PTC_OMNI_LIVE_FOOD_FL: No
PTC_OMNI_BRAND_PRIMARY: Kaytee
parentCatEntryIDAsString: 865023
PTC_OMNI

In [12]:
# Профильтруем датасет, развернув колонку метаданных и оставив только нужные метаданные и facets
metadata = ['url', 'image_url', 'itemname', 'group_ids']
facets = ['How to get it', 'Primary Brand', 'Pet Type', 'Primary Pet Type']
data_items_cleaned = process_data.clean_data_items(data_items, metadata = metadata, facets = facets)
data_items_cleaned.show(truncate=False)

+-----------+-------------------------------------------------------------------------------+-------------------------------------------------------------------------------+-------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------+-----------------------+--------------+----------------+
|customer_id|name                                                                           |name_lower               

In [13]:
# Сохраним презентадельный вид датасета
cleaned_data_folder_path = WORKING_DIRECTORY+'data/cleaned_data/'
output_name = 'data_items'
process_data.save_dataset_parquet(data_items_cleaned, cleaned_data_folder_path, output_name)

Папка /content/drive/MyDrive/Colab Notebooks/diploma/data/cleaned_data/ уже существует.
Датасет сохранён по пути /content/drive/MyDrive/Colab Notebooks/diploma/data/cleaned_data/data_items.


In [14]:
# Приведем датасет к формату SASRec
features = ['group_ids_intersect', 'How_to_get_it', 'Primary_Brand', 'Primary_Pet_Type']
features_final_names = ['Category', 'Delivery', 'Brand', 'Pet']
data_items_processed = process_data.items_to_sasrec_format(data_items_cleaned, features, features_final_names)
# print(f"Размер датасета: ({data_items_processed.count()}, {len(data_items_processed.columns)})")
# data_items_processed.show()

In [15]:
# Сохраним датасет формата SASRec
sasrec_data_folder_path = WORKING_DIRECTORY+'data/sasrec_format/'
output_name = 'items'
process_data.save_dataset_parquet(data_items_processed, sasrec_data_folder_path, output_name)
# data_items_processed = spark.read.parquet(sasrec_data_folder_path+output_name)

Папка /content/drive/MyDrive/Colab Notebooks/diploma/data/sasrec_format/ уже существует.
Датасет сохранён по пути /content/drive/MyDrive/Colab Notebooks/diploma/data/sasrec_format/items.


In [16]:
# Проведем тесты на правильность сборки датасета для SASRec
metadata_original_names = ['group_ids']
facets_original_names = ['How to get it', 'Primary Brand', 'Primary Pet Type']
features_final_names = ['Category', 'Delivery', 'Brand', 'Pet']
test_items_sasrec(data_items_processed, data_items, metadata_original_names, facets_original_names, features_final_names)

✅ В датасете правильные колонки и у них правильные типы данных
✅ В датасете нет пропущенных значений
✅ Тест на множество товаров пройден


100%|██████████| 10/10 [25:45<00:00, 154.58s/it]


✅ Признаки товаров в датасете совпадают с каталогом


In [17]:
common_tests_sasrec(df_actions=data_conversions_processed, df_items=data_items_processed)

18640 32001
✅ В финальном датасете товаров есть 99.9% всех необходимых товаров
